In [ ]:
import sys
sys.path.append('..')

from astar.astar import *
from scipy.constants import pi

import numpy as np

In [ ]:
START=[300,300]
STOP=[2000,1000]

x_max=3000.0
y_max=2000.0
nb_x=30
nb_y=20

robotSize= np.array([300,200,np.sqrt(300**2+200**2),] )

In [ ]:
## Création de la Map du pathfinding
Map=Pathfinder(x_max, y_max, nb_x, nb_y, 150., 500.)
# ATTENTION LES TYPES DOIVENT ÊTRE RESPECTER : CORRIGER CELA DANS LE SOURCE astar.py

In [ ]:
import tkinter as tk
import tkinter.ttk as ttk
import PIL.Image
import PIL.ImageTk

CANVAS_WIDTH = 600
CANVAS_HEIGHT = 400

class Application(tk.Frame):
    def __init__(self, mapDxy=20, mapResize=5., master=None):
        super().__init__(master)
        
        self.mapDxy = int(mapDxy)
        self.mapResize = mapResize
        self.nbX = int(CANVAS_WIDTH/mapDxy)
        self.nbY = int(CANVAS_HEIGHT/mapDxy)
        self.mapLstSquare=[]
        self.mapLstPath = []
        # Table des obstacles
        self.mapTblNode = np.zeros((self.nbX,self.nbY))
        self.mapIdSquare = None
        
        self.master.title('Robot')
        # Force la taille de la fenetre
        self.master.geometry("800x480+0+0")
        #supprime la bar de titre de la fenetre
        #self.master.overrideredirect(tk.TRUE)
        
        # Création notebook (onglet)
        self.note = ttk.Notebook(self.master)
        self.tab1 = ttk.Frame(self.note)
        self.tab2 = ttk.Frame(self.note)
        self.note.add(self.tab1, text = "Map",compound=tk.TOP)#, image=scheduledimage)
        self.note.add(self.tab2, text = "Monitor")

        # Création d'un widget Canvas (zone graphique)
        self.canevas = tk.Canvas(self.tab1, width=CANVAS_WIDTH, height=CANVAS_HEIGHT, bg='white')

        # La méthode bind() permet de lier un événement avec une fonction
        self.canevas.bind('<Button-1>',self.map_clic) # évévement clic gauche (press)
        self.canevas.bind('<B1-Motion>',self.map_drag) # événement bouton gauche enfoncé (hold down)
        self.canevas.bind('<ButtonRelease-1>',self.map_release) # événement bouton gauche relaché

        self.canevas.focus_set()
        self.note.pack()
        self.canevas.pack(side=tk.LEFT, padx=5, pady=5)
        
        self.b1 = ttk.Button(self.tab1, text ='Save', command = self.map_save)
        self.b2 = ttk.Button(self.tab1, text ='Erase', command = self.map_erase)
        self.b3 = ttk.Button(self.tab1, text ='Reload', command = self.map_reload, style="C.TButton")
        self.b4 = ttk.Button(self.tab1, text ='Find', command = self.map_find)
        self.b1.pack(padx=5,pady=5)
        self.b2.pack(padx=5,pady=5)
        self.b3.pack(padx=5,pady=5)
        self.b4.pack(padx=5,pady=5)

        self.map_reload()
            
        # Création d'un widget Button (bouton Quitter)
        ttk.Button(self.master, text ='Quitter', command = self.master.destroy).pack(side=tk.LEFT,padx=5,pady=5)
        
        

    def map_release(self, event):
        if (self.mapIdSquare is not None) and (self.mapIdSquare >1):
            # position du pointeur de la souris
            X = int(event.x/self.mapDxy)
            Y = int(event.y/self.mapDxy)
            # Ajout d'un element dans la Table des obstacles
            self.mapTblNode[X][Y] = 1
    
    def map_clic(self, event):
        """ Gestion de l'événement Clic gauche """
        self.mapIdSquare = None

        # position du pointeur de la souris
        X = int(event.x/self.mapDxy)
        Y = int(event.y/self.mapDxy)
        # Retrait d'un element dans la Table des obstacles
        # L'ajout se fait lors du release du clic gauche
        self.mapTblNode[X][Y] = 0
        X *= self.mapDxy
        Y *= self.mapDxy

        if not(self.mapLstSquare):
            self.mapLstSquare += [self.canevas.create_rectangle(X,Y,X+self.mapDxy,Y+self.mapDxy,fill='#999999',width=0)]
            self.mapIdSquare = 0
        else:
            canvas = event.widget
            item_ids = self.canevas.find_closest(event.x, event.y)
            for iC, Carre in enumerate(self.mapLstSquare):
                # coordonnées de l'objet
                coords=self.canevas.coords(Carre)
                [xmin,ymin,xmax,ymax] = coords[:4]
                if xmin<=X<xmax and ymin<=Y<ymax:
                    self.mapIdSquare = iC
                    return
            self.mapLstSquare += [self.canevas.create_rectangle(X,Y,X+self.mapDxy,Y+self.mapDxy,fill='#999999',width=0)]
            self.mapIdSquare = iC+1

    def map_drag(self, event):
        """ Gestion de l'événement bouton gauche enfoncé """
        X = int(event.x/self.mapDxy)*self.mapDxy
        Y = int(event.y/self.mapDxy)*self.mapDxy

        if self.mapIdSquare is not None:
            Square = self.mapLstSquare[self.mapIdSquare]
            if X<0 or X>CANVAS_WIDTH or Y<0 or Y>CANVAS_HEIGHT:
                self.canevas.delete(Square)
                self.mapLstSquare.pop(self.mapIdSquare)
                self.mapIdSquare = None
            else:
            # mise à jour de la position de l'objet (drag)
                self.canevas.coords(Square,X,Y,X+self.mapDxy,Y+self.mapDxy)
                #self.mapIdSquare = None

    def map_erase(self):
        """ Efface la zone graphique """
        self.mapClickDetection = None
        self.mapLstSquare = []
        self.mapTblNode = np.zeros((self.nbX,self.nbY))
        self.canevas.delete(tk.ALL)
        #self.mapLstSquare = [self.canevas.create_rectangle(START[0]/self.mapResize,START[1]/self.mapResize,
        #                                                   START[0]/self.mapResize+self.mapDxy,START[1]/self.mapResize+self.mapDxy,
        #                                                   fill='green',width=0)]
        
        alpha = pi/4
        rob = robotSize/self.mapResize/2
        phi = np.arccos(rob[0]/rob[2])
        x0 = START[0]/self.mapResize + rob[2]*np.cos(alpha+phi)
        y0 = START[0]/self.mapResize + rob[2]*np.sin(alpha+phi)
        x1 = START[0]/self.mapResize + rob[2]*np.cos(alpha+pi-phi)
        y1 = START[0]/self.mapResize + rob[2]*np.sin(alpha+pi-phi)
        x2 = START[0]/self.mapResize + rob[2]*np.cos(alpha+pi+phi)
        y2 = START[0]/self.mapResize + rob[2]*np.sin(alpha+pi+phi)
        x3 = START[0]/self.mapResize + rob[2]*np.cos(alpha-phi)
        y3 = START[0]/self.mapResize + rob[2]*np.sin(alpha-phi)
        
        pts = [x0,y0,x1,y1,x2,y2,x3,y3]
        self.mapLstSquare = [self.canevas.create_polygon(pts, fill="#42BCFF")]
        
        
        self.mapLstSquare+= [self.canevas.create_rectangle(STOP[0]/self.mapResize,STOP[1]/self.mapResize,
                                                           STOP[0]/self.mapResize+self.mapDxy,STOP[1]/self.mapResize+self.mapDxy,
                                                           fill='red',width=0)]
        for i in range(0,self.nbY*self.mapDxy,self.mapDxy):
            self.canevas.create_line(0,i,CANVAS_WIDTH,i, fill='#999999')
        for i in range(0,self.nbX*self.mapDxy,self.mapDxy):
            self.canevas.create_line(i,0,i,CANVAS_HEIGHT, fill='#999999')
            
        
            
    
    def map_save(self):
        np.savetxt("obstacle.csv", self.mapTblNode, delimiter=";")
    
    def map_reload(self):
        self.map_erase()
        try:
            self.mapTblNode = np.genfromtxt("obstacle.csv", delimiter=";")
        except:
            pass
        for Node in np.argwhere(self.mapTblNode>0):
            X = Node[0]*self.mapDxy
            Y = Node[1]*self.mapDxy
            self.mapLstSquare += [self.canevas.create_rectangle(X,Y,X+self.mapDxy,Y+self.mapDxy,fill='#999999',width=0)]
        
    def map_find(self):    
        for i in range(len(self.mapLstPath)):
            self.canevas.delete(self.canevas.delete(self.mapLstPath[i]))
        
        START = self.canevas.coords(self.mapLstSquare[0])[:2]
        STOP = self.canevas.coords(self.mapLstSquare[1])[:2]
        print(STOP)
        # Ajout de de la carte comme obstacles à la Map astar
        ObsMap=np.zeros((self.nbX,self.nbY))+600.
        for ix_obs, obsList in enumerate(self.mapTblNode):
            for iy_obs, obs in enumerate(obsList):
                if obs > 0:
                    for j in range(-Map.iObsMax_y,Map.iObsMax_y):
                        for i in range(-Map.iObsMax_x,Map.iObsMax_x):
                            iy=iy_obs+j
                            ix=ix_obs+i
                            if (0<=ix<self.nbX) & (0<=iy<self.nbY):
                                NewDist = sqrt((ix-ix_obs)**2*Map.delta_x**2+(iy-iy_obs)**2*Map.delta_y**2)
                                if NewDist < ObsMap[ix,iy]:
                                    ObsMap[ix,iy] = NewDist   
        Map.setObstacle(ObsMap)
        # Calcul du plus court Chemin
        
        Map.clean()
        NodeFin = Map.pathfinding(int(START[0]*self.mapResize),int(START[1]*self.mapResize),0,
                                  int(STOP[0]*self.mapResize),int(STOP[1]*self.mapResize),
                                  VtsMax=500.,VtsAng=10.)
        TableX=[]
        TableY=[]
        TableAng=[]
        TableTpsTrajet=[]
        TableObs=[]
        if NodeFin is not None:
            NodeFin.getParcours(TableX,TableY,TableAng,TableTpsTrajet,TableObs)


            XXX=np.array(TableX[1:-1])/self.mapResize
            YYY=np.array(TableY[1:-1])/self.mapResize
            for i, X in enumerate(XXX):
                self.mapLstPath += [self.canevas.create_rectangle(X,YYY[i] ,X+self.mapDxy,YYY[i]+self.mapDxy,fill='blue',width=0)]

            DeltaAng=np.convolve(TableAng,[1,-1],mode="same")
            PointAnguleux=np.where((DeltaAng>=pi/4.0)|(DeltaAng<=-pi/4.0))
            PointAnguleux=PointAnguleux[0]
            j=0
            NbPt=len(PointAnguleux)*2
            for i in PointAnguleux:
                String="PstCons %s 0 %.4s %s %s %s 4 %s 0.1 10.0 10.0 \n" % (NbPt, TableAng[i], TableX[i], TableY[i],'50','0')
                #String="TuRn %.4s %s %s %s \n" % (j, TableAng[i],Vmax,wait)
                #ser.write(String) # pour un envoie de la chaine de caratères sur par port COM
                print(String[:-1])
                j=j+1
                String="PstCons %s 0 %.4s %s %s %s 3 %s 0.05 100.0 100.0 \n" % (NbPt, TableAng[i], TableX[i], TableY[i],'50','0')
                #String="GoTo %s %s %s %s %s \n" % (j, TableX[i], TableY[i],Vmax,wait)
                #ser.write(String) # pour un envoie de la chaine de caratères sur par port COM
                print(String[:-1])
                j=j+1
        else:
            print("No path")
        
        return

In [ ]:
root = tk.Tk()
app = Application(x_max/nb_x/5,5,master=root)
app.mainloop()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Obs=Map.getTable()
plt.pcolor(Map.X,Map.Y,Obs.T)

In [ ]:
Map.clean()
NodeFin = Map.pathfinding(0.,0.,0.,2000.,1000.,VtsMax=50.,VtsAng=1.)
print(NodeFin)